## Fluídos Conformes

Código para testear la estabilidad alrededor de una solución de equilibrio.

Las ecuaciones son:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k
$$

$$
c^j = C^j(f) = (e, S, c_i)
$$

Lo que hacemos es tomar el jacobiano del flujo,

$\frac{\partial F^i}{\partial f^j}$ donde $f^j$ son las variables no conservativas para las cuales tenemos las expresiones del Flujo. 
A estas las multiplicamos por $[\frac{\partial c^k}{\partial f^j}]^{-1}$.
Tenemos así 

$$
P^{il}{}_j := \frac{\partial F^{il}}{\partial c^j} = \frac{\partial F^{il}}{\partial f^k}\frac{\partial f^k}{\partial c^j}.
$$

Evaluamos todo en el vector $(-1,0,0,0,0)$ o sea estamos en el frame con velocidad cero y todas las variables fuera de equilibrio también cero.

Lo mismo hacemos con la fuente. 

$$
II^i{}_j = \frac{\partial I^i}{\partial f^k} \frac{\partial f^k}{\partial c^j}
$$

Tenemos así el sistema linearizado en el punto de equilibrio en las variables conservativas. Multiplicamos $P^i{}_j$ por un número complejo cualquiera y para todos ellos los autovalores debieran tener parte real negativa o nula. Pero no es así...


Alternativamente podemos diferenciar $c^i$ con respecto a los $f^j$ para obtener,

$$
\partial_t c^j = \frac{\partial c^j}{\partial f^i} \partial_t f^i
$$

y por lo tanto,

$$
\partial_t f^i = \frac{\partial c^j}{\partial f^i}^{-1} \frac{\partial F^{lj}}{\partial f^k}\partial_l f^k := \tilde{P}^i{}_k\partial_l f^k
$$

A las que simplemente sumamos el término $\frac{\partial c^j}{\partial f^i}^{-1}\frac{\partial I^j}{\partial f^k}$ para tener el sistema linearizado con respecto a las $f^i$.

In [ ]:
using Symbolics
using LinearAlgebra
using Latexify

In [ ]:
function Flux(flu,par)
    χ = par
    #flux = zeros(5)
    μ = flu[1]  
    T = (abs(μ))^(-1//2) 
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
#    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    Q = 10χ₁ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    #flux[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    #flux[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    #flux[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #flux[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4
    #flux[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4
    #return -flux[:]
    return [4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v; 
            4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ;
            χ₁*γ*v*(6*γ^2 - 1)/μ^3/T   - 12χ₂*(v*(6γ^2 - 1)*x1   + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4;
            #χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
            χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v^2+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
            3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 + 3*(1+6γ^2*v^2)*x2      + 3v*x3)/μ^4]
end

Chequeamos con los valores de Marcelo. 

In [ ]:
χ = [- 1.; - 2.; - 10.]
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
Flu = [5.921317694643806; 6.02302807825841; 3.713391378258412;  4.136735467078638; 3.444079555898864]

Flux(flu,χ) - Flu;

Calculamos el Jacobiano

In [ ]:
@variables f[1:5], c[1:5], p[1:3]#, fl[1:5]

JFS = Symbolics.jacobian(Flux(f,p),f);
JF_exp = Symbolics.build_function(JFS, f, p);
JFac = eval(JF_exp[1]);

Definimos algunos valores para testear

In [ ]:
@variables x1, x2, x3

flu=[-1.;0;0;0;0];
con=ones(5)
χ=[-1.,-0.5,-5.]

El Jacobiano en equilibrio.

In [ ]:
JFac(flu,p)

Ahora traemos el Jacobiano de la transformación entre las variables fluido/conservativas.

In [ ]:
include("inversion_ext.jl");

In [ ]:
@variables χ₂ μ γ v

A = (-12 * χ₂/μ^4)*[3(2γ^2 - 1)  3v*(6γ^2 - 1)    3v^2 ;
                v*(6γ^2 - 1)  (6γ^2*(1 + 2v^2) - 1)  v*(v^2 + 2) ;
                (6γ^2 * v^2 + 1)  v*(6γ^2*(2 + v^2) - 1)  (2v^2 + 1)]

A[1,2]


Para posterior uso calculamos las variables conservativas correspondientes al estado de equilibrio. Esto no me lo esperaba, pensaba que me darían c1=c2=c3=0!

In [ ]:
con = F(flu,zeros(5), p);

Jac(flu,con,p)

In [ ]:
P(flu,c,p) = Jac(flu,c,p) \ JFac(flu,p)

In [ ]:
flu = [-1;0.9;0;0;0]
M0=simplify.(substitute.(P(flu,c,p), (Dict(p[2] => 0),)))

In [ ]:
P(flu,c,p);

In [ ]:
18.75/3.75*36

In [ ]:
P(flu,c,p)[4,3];

In [ ]:
P(flu,c,p)';

In [ ]:
Ratio = P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3];

In [ ]:
#par_f = [-1,-0.5,-10]
#substitute(P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3], (Dict(p => par_f),))


In [ ]:
simplify(12*(10p[1]*(-36p[3] - 3.75p[2]^2/p[1]) + 6p[2]*(6.25p[2] - 60p[3])) / (6.25p[2] - 60p[3]) / p[1])

In [ ]:
par_f = [-1,-0.5,-10]
P(flu,c,par_f)'[4,1]/P(flu,c,par_f)'[4,3]

Miramos primero el caso donde las variables están desacopladas, es decir con $\chi_1=0$.

In [ ]:
P(flu,c,[-1,-0,-2])

Vemos que se corresponde con la siguiente matriz racional.

In [ ]:
M = [0 2//3 0 0 0;
    1//2 0 -1//4 0 -1//4;
    0 0 0 5//3 0;
    0 0 1//5 0 1//5;
    0 0 0 4//3 0];

In [ ]:
P(flu,c,[-1,0,-1]) - M

In [ ]:
eigvals(M)

In [ ]:
1/0.7745966692414837^2 - 5/3

In [ ]:
1/0.5773502691896258^2 - 3

In [ ]:
eigvals(P(flu,c,[-1,-0.5,-1]))

Hacemos lo mismo para la fuente.

In [ ]:
function Is(flu, χ, ξ)
    #(χ, ξ) = par 
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= -χ[2] # lo hacemos positivo
    κ = χ₀*ξ[1]/χ₁^2 # OK 
    λ = χ₀*ξ[2]/χ₁^2  # OK
    η = χ₀*ξ[3]/χ₁^2
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    γ = (1. - v^2)^(-1//2)
    #Is[1] = 0.
    #Is[2] = 0.
    #Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
    #Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
    #Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
    #return Is[:]      #(1 - ℯ^(-5. *t))
    return [0.;
            0.;
            -(3//10*(γ^2-1//4)*x1/(γ*λ)   + 2γ*v*x2/κ      - v^2*x3/(γ*η))/μ^4/T;
            -(3//10*γ*v*x1/λ              + γ*(v^2+1)*x2/κ - v*x3/(γ*η))/μ^4/T;
            -(3//10*(γ^2*v^2+1//4)*x1/λ/γ + 2γ*v*x2/κ      - x3/(γ*η))/μ^4/T;
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*η);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*η);
            #-2//5*(γ^2*v^2+1//4)*T*x1/η/γ - 2γ*v*x2/T/κ + T*x3/(γ*η)
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            #-2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
            ]
end

La chequeamos con valores de Marcelo.

In [ ]:
t=0
flu = [- 5.; 0.0; 2.1; 0.5; 5.1]
par = [-1., -2., -10.]
par_s = [1/15π, 1/1π, 1/1π]
I_c = [0.; 0.; - 0.09488575328013785; - 0.12155655385334033; - 0.12140081195218293]
Is(flu,par,par_s) - I_c;

In [ ]:
@variables q[1:3]  
Is(f,p,q);

In [ ]:
JSS = Symbolics.jacobian(Is(f,p,q),f);

In [ ]:
JS_exp = Symbolics.build_function(JSS, f, p,q);

In [ ]:
JSac = eval(JS_exp[1]);

In [ ]:
flu=[1.;0.3;0;0;0];
con=ones(5)
#par_f = [-1,-2,-10]
par_f = [-1,0,-4]
#JSac(flu,par_f,par_s)
#eigvals(JSac(flu,par_f,par_s)) #esta no es la matríz correcta para sacarle los autovalores
eigvals(P(flu,c,par_f))

Si $\chi_1=0$ los autovalores (velocidades) no dependen de los otros $\chi$ y hay uno que crece muchísimo cuando $v \to 1$. Eso trae problemas para evolucionar las ecuaciones por el CFL.

In [ ]:
II(flu,c,p,q) = Jac(flu,c,p) \ JSac(flu,p,q)

In [ ]:
II(flu,c,p,q);

In [ ]:
flu=[1.;0.9;0;0;0];
#con=ones(5)
par_f = [-1.,-1.,-10.]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, 0.01/1π]
con = F(flu,zeros(5), par_f);
function S(k,s,flu,par_f,par_s) 
    con = F(flu,zeros(5), par_f);
    return eigvals(k .* P(flu,con,par_f) + s .*II(flu,con,par_f,par_s))
end

function par_ff(i,j,N,M,p2i,p2f,p3i,p3f)
    par_f[1] = -1
    par_f[2] = p2i + (i-1)/(N-1)*(p2f - p2i)
    par_f[3] = p3i + (j-1)/(M-1)*(p3f - p3i)
    return par_f
end

In [ ]:
using Plots
#using Gadfly
#unicodeplots()
gr()

In [ ]:
tol = 10^(-14)
N = 501
M = 500
Max = zeros(N,M)
p2i = -1
p2f = 1
p3i = -10
p3f = 10

for i in 1:N
    for j in 1:M
        par_f = par_ff(i,j,N,M,p2i,p2f,p3i,p3f)
        R = real.(S(im,0,flu,par_f,par_s))
        I = imag.(S(im,0,flu,par_f,par_s))
        if R'*R < tol
            MM = maximum(abs,I)
            if MM <= 1 
            Max[i,j] = MM
            end
        else
            Max[i,j] = 2
        end
    end
end
    

In [ ]:
T = [1 2 3;4 5 6]
display(heatmap(T'))

In [ ]:
T[1,3]


In [ ]:
heatmap(Max')

In [ ]:
i=100 # chi_1
j=200 # chi_2
par_f = par_ff(i,j,N,M,p2i,p2f,p3i,p3f)
S(im,0,flu,par_f,par_s)
#par_f

In [ ]:
MinRS = zeros(N,M)

for i in 1:N
    for j in 1:M
        par_f = par_ff(i,j,N,M,p2i,p2f,p3i,p3f)
        E = S(im,1,flu,par_f,par_s)
        R = real.(E)
        I = imag.(E)
        MM = maximum(R)
            if MM <= 0 
            MinRS[i,j] = MM
            end
    end
end

In [ ]:
Plots.spy(MinRS)

Vemos que un par de autovalores tiene parte real positiva. 

In [ ]:
eigvecs(II(flu,c,par_f,par_s))

In [ ]:
par_f = [-1,-1,-10]
E = eigvecs(10*im .* P(flu,c,par_f) - 0. * II(flu,c,par_f,par_s));

In [ ]:
[E[:,1]; E[:,5]]

In [ ]:
[E[:,2]; E[:,4]]

In [ ]:
E[:,3]

In [ ]:
Ei = inv(E)

In [ ]:
Ei[3,:]

In [ ]:
Ei[5,:]'*E[:,5]

In [ ]:
ET = eigvecs(10*im .* P(flu,c,par_f)');

In [ ]:
[ET[:,1]; ET[:,5]];

In [ ]:
[ET[:,2]; ET[:,3]];

In [ ]:
ET[:,4] - ET[1,4]/Ei[4,1] .* Ei[4,:];

In [ ]:
1/(Ei[3,1]/Ei[3,3])

In [ ]:
1/(Ei[3,1]/Ei[3,5])

 [-1,-1,-1] -> 16/5, 4  $\frac{4\chi_2}{\chi_1}(4/5,1)$
 [-1,-2,10] -> -16, 20
 [-1,-0.5,-1] -> -64, 80

In [ ]:
4χ[2]/χ[1]*[4/5;1]

In [ ]:
function coc(par_f)
    return 4par_f[3]/par_f[2]*[-4/5 ; 1]
end

In [ ]:
coc([-1,-1,-10])

In [ ]:
coc([-1,-0.5,-10])

Alternativamente podemos quedarnos con las expresiones en derivadas del los campos del fluido.
Es decir:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k + I^j(j)
$$

In [ ]:
flu=[1.;0.0;0;0;0];
#con=ones(5)
par_f = [-1.,1.,-10.]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, 1/1π, -0.1/1π]
con = F(flu,zeros(5), par_f);
PP = inv(Jac(flu,con,par_f))*(1.0im*JFac(flu,par_f) + 1.0.*JSac(flu,par_f,par_s));

In [ ]:
eigvals(PP)

In [ ]:
Et = eigvecs(PP);

In [ ]:
Ett = inv(Et);

In [ ]:
H = adjoint(Ett)*Ett
#latexify(H)

In [ ]:
H*PP - adjoint(PP)*H

In [ ]:
eigvals(H)

In [ ]:
eigvals(H*(1.0im.* JFac(flu,par_f) - 1.0.*JSac(flu,con,par_s)))

In [ ]:
H*JSac(flu,con,p)

In [ ]:
1/1.333333

In [ ]:
200/27